In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE

In [2]:
conn = sqlite3.connect('spotify.sqlite')
cursor = conn.cursor()
query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(query)
tables = cursor.fetchall()
print("List of tables in the database:")
for table in tables:
    print(table[0])

List of tables in the database:
albums
artists
audio_features
genres
r_albums_artists
r_albums_tracks
r_artist_genre
r_track_artist
tracks
artists_indexed
albums_indexed
tracks_indexed
audio_features_indexed


In [3]:
query = "SELECT * FROM tracks"
encoding = "latin1"
conn.text_factory = lambda x: str(x, encoding)
track_df = pd.read_sql_query(query, conn)
print(track_df)

                             id  disc_number  duration  explicit  \
0        1dizvxctg9dHEyaYTFufVi            1    275893         1   
1        2g8HN35AnVGIk7B8yMucww            1    252746         1   
2        49pnyECzcMGCKAqxfTB4JZ            3    315080         0   
3        4E5IFAXCob6QqZaJMTw5YN            1    240800         1   
4        1gSt2UlC7mtRtJIc5zqKWn            2    203666         0   
...                         ...          ...       ...       ...   
8741667  12S1TA13KXl13sCQZgtI5y            1    198000         0   
8741668  134MLEMX0AWp5kbMMf90f8            1    191360         0   
8741669  13WVJaMn0ztUCU8UFCKQFO            1    272572         0   
8741670  11SfAvTgnEEYy4Ef0fyFpE            1    237200         1   
8741671  13EW51DKru044OJDJ9Bm8o            1    153147         1   

               audio_feature_id  \
0        1dizvxctg9dHEyaYTFufVi   
1        2g8HN35AnVGIk7B8yMucww   
2        49pnyECzcMGCKAqxfTB4JZ   
3        4E5IFAXCob6QqZaJMTw5YN   
4       

In [4]:
track_sorted_df = track_df.sort_values(by='popularity', ascending=False)
track_sorted_df

,id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable
802846,4iJyoBOLtHqaGxP12qzhQI,1,198081,1,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),,12,100,NaN
109193,7lPN2DXiMsVn7XUKtOW1CS,1,242013,1,7lPN2DXiMsVn7XUKtOW1CS,drivers license,,1,100,NaN
6202114,3VqeTFIvhxu3DIe4eZVzGq,1,164441,0,3VqeTFIvhxu3DIe4eZVzGq,Butter,,1,100,NaN
4025933,67BtfxlNbhBmCDR2L2l8qd,1,137875,1,67BtfxlNbhBmCDR2L2l8qd,MONTERO (Call Me By Your Name),https://p.scdn.co/mp3-preview/fee76abaab413004...,1,99,NaN
613929,3BZEcbdtXQSo7OrvKRJ6mb,1,137875,1,3BZEcbdtXQSo7OrvKRJ6mb,MONTERO (Call Me By Your Name),,1,99,NaN
...,...,...,...,...,...,...,...,...,...,...
4668250,0QLCx9hQIILb8OmrqeF75q,1,327610,1,0QLCx9hQIILb8OmrqeF75q,Dame Micro,,9,0,NaN
4668247,1xSwsAa29i5iHQlIUDNVY7,1,261154,1,1xSwsAa29i5iHQlIUDNVY7,No Tiene Huesos,,10,0,NaN
4668245,1fyCuwobAvxr8q0TU8jdNs,1,288740,1,1fyCuwobAvxr8q0TU8jdNs,Gracias,,3,0,NaN
323081,18qyGR3v4026xRQQnDru8o,1,206336,0,18qyGR3v4026xRQQnDru8o,Eenie Meenie (George Gershwin Arrangement),https://p.scdn.co/mp3-preview/bc2087efdba03530...,7,0,NaN


In [5]:
top_df = track_sorted_df.head(10000)
top_df

,id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable
802846,4iJyoBOLtHqaGxP12qzhQI,1,198081,1,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),,12,100,NaN
109193,7lPN2DXiMsVn7XUKtOW1CS,1,242013,1,7lPN2DXiMsVn7XUKtOW1CS,drivers license,,1,100,NaN
6202114,3VqeTFIvhxu3DIe4eZVzGq,1,164441,0,3VqeTFIvhxu3DIe4eZVzGq,Butter,,1,100,NaN
4025933,67BtfxlNbhBmCDR2L2l8qd,1,137875,1,67BtfxlNbhBmCDR2L2l8qd,MONTERO (Call Me By Your Name),https://p.scdn.co/mp3-preview/fee76abaab413004...,1,99,NaN
613929,3BZEcbdtXQSo7OrvKRJ6mb,1,137875,1,3BZEcbdtXQSo7OrvKRJ6mb,MONTERO (Call Me By Your Name),,1,99,NaN
...,...,...,...,...,...,...,...,...,...,...
1944958,3WcWRr8wIkwPQOTYBWjEGf,1,244306,0,3WcWRr8wIkwPQOTYBWjEGf,Potevi fare di piÃ¹,https://p.scdn.co/mp3-preview/179164f8b2612b21...,1,67,NaN
1945150,1F1QmI8TMHir9SUFrooq5F,1,215384,1,1F1QmI8TMHir9SUFrooq5F,Buff Baby,https://p.scdn.co/mp3-preview/df80e5a8c9e38a8e...,2,67,NaN
2987221,7FIkmLlwTJCK3CXrNIwXXX,1,193044,1,7FIkmLlwTJCK3CXrNIwXXX,Bis ich tot bin,,1,67,NaN
72943,69QIVIjPMnI1ABLBqh9P1b,1,260666,0,69QIVIjPMnI1ABLBqh9P1b,Twisted,https://p.scdn.co/mp3-preview/d7373f83c11623d3...,1,67,NaN


In [6]:
query = "SELECT * FROM audio_features"
encoding = "latin1"
conn.text_factory = lambda x: str(x, encoding)
features_df = pd.read_sql_query(query, conn)
print(features_df)

                             id  acousticness  \
0        2jKoVlU7VAmExKJ1Jh3w9P        0.1800   
1        4JYUDRtPZuVNi7FAnbHyux        0.2720   
2        6YjKAkDYmlasMqYw73iB0w        0.0783   
3        2YlvHjDb4Tyxl4A1IcDhAe        0.5840   
4        3UOuBNEin5peSRqdzvlnWM        0.1700   
...                         ...           ...   
8740038  7xq5qTqdCvTQJomW9I5pSZ        0.1430   
8740039  6ClotRUApTN25L3OnkzQjb        0.0417   
8740040  7mZL8mtlavFzAzIlIfOMv2        0.2570   
8740041  08ITn8qaxbnEyk3N08IC47        0.0361   
8740042  5Vqgrz2M1m49PRh2eR9et7        0.1510   

                                              analysis_url  danceability  \
0        https://api.spotify.com/v1/audio-analysis/2jKo...         0.893   
1        https://api.spotify.com/v1/audio-analysis/4JYU...         0.520   
2        https://api.spotify.com/v1/audio-analysis/6YjK...         0.918   
3        https://api.spotify.com/v1/audio-analysis/2Ylv...         0.877   
4        https://api.spotify.co

In [7]:
track_and_features_df = pd.merge(top_df, features_df, on='id', how='inner')

In [8]:
track_and_features_df

,id,disc_number,duration_x,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,4iJyoBOLtHqaGxP12qzhQI,1,198081,1,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),,12,100,NaN,...,0.696,0.000000,0,0.4200,-6.181,1,0.1190,90.029999,4,0.464
1,7lPN2DXiMsVn7XUKtOW1CS,1,242013,1,7lPN2DXiMsVn7XUKtOW1CS,drivers license,,1,100,NaN,...,0.436,0.000013,10,0.1050,-8.761,1,0.0601,143.873993,4,0.132
2,3VqeTFIvhxu3DIe4eZVzGq,1,164441,0,3VqeTFIvhxu3DIe4eZVzGq,Butter,,1,100,NaN,...,0.459,0.000000,8,0.0906,-5.187,1,0.0948,109.997002,4,0.695
3,67BtfxlNbhBmCDR2L2l8qd,1,137875,1,67BtfxlNbhBmCDR2L2l8qd,MONTERO (Call Me By Your Name),https://p.scdn.co/mp3-preview/fee76abaab413004...,1,99,NaN,...,0.508,0.000000,8,0.3840,-6.682,0,0.1520,178.817993,4,0.758
4,3BZEcbdtXQSo7OrvKRJ6mb,1,137875,1,3BZEcbdtXQSo7OrvKRJ6mb,MONTERO (Call Me By Your Name),,1,99,NaN,...,0.508,0.000000,8,0.3840,-6.682,0,0.1520,178.817993,4,0.758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,3WcWRr8wIkwPQOTYBWjEGf,1,244306,0,3WcWRr8wIkwPQOTYBWjEGf,Potevi fare di piÃ¹,https://p.scdn.co/mp3-preview/179164f8b2612b21...,1,67,NaN,...,0.391,0.000000,4,0.0906,-8.317,1,0.0559,123.705002,4,0.162
9995,1F1QmI8TMHir9SUFrooq5F,1,215384,1,1F1QmI8TMHir9SUFrooq5F,Buff Baby,https://p.scdn.co/mp3-preview/df80e5a8c9e38a8e...,2,67,NaN,...,0.765,0.000000,5,0.1010,-6.047,1,0.0571,156.039993,4,0.512
9996,7FIkmLlwTJCK3CXrNIwXXX,1,193044,1,7FIkmLlwTJCK3CXrNIwXXX,Bis ich tot bin,,1,67,NaN,...,0.503,0.000000,0,0.2150,-7.906,0,0.0816,139.011993,4,0.388
9997,69QIVIjPMnI1ABLBqh9P1b,1,260666,0,69QIVIjPMnI1ABLBqh9P1b,Twisted,https://p.scdn.co/mp3-preview/d7373f83c11623d3...,1,67,NaN,...,0.514,0.000000,11,0.2480,-9.759,1,0.0466,175.848999,4,0.822


In [9]:
print(track_and_features_df.columns)

Index(['id', 'disc_number', 'duration_x', 'explicit', 'audio_feature_id',
       'name', 'preview_url', 'track_number', 'popularity', 'is_playable',
       'acousticness', 'analysis_url', 'danceability', 'duration_y', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')


In [10]:
query = "SELECT * FROM r_track_artist"
encoding = "latin1"
conn.text_factory = lambda x: str(x, encoding)
r_track_artist_df = pd.read_sql_query(query, conn)
print(r_track_artist_df)

                        track_id               artist_id
0         2jKoVlU7VAmExKJ1Jh3w9P  4tujQJicOnuZRLiBFdp3Ou
1         2jKoVlU7VAmExKJ1Jh3w9P  2VX0o9LDIVmKIgpnwdJpOJ
2         2jKoVlU7VAmExKJ1Jh3w9P  3iBOsmwGzRKyR0vs2I61xP
3         2jKoVlU7VAmExKJ1Jh3w9P  22qf8cJRzBjIWb2Jc4JeOr
4         4JYUDRtPZuVNi7FAnbHyux  4akj4uteQQrrGxhX9Rjuyf
...                          ...                     ...
11840397  6ClotRUApTN25L3OnkzQjb  3y9hQYlwNKqhdybIJTFzTr
11840398  7mZL8mtlavFzAzIlIfOMv2  5e8vdggiGg9glSpuBgcaiv
11840399  7mZL8mtlavFzAzIlIfOMv2  6NgwMiz1Dwsktnn8p8yujE
11840400  08ITn8qaxbnEyk3N08IC47  4NEA8zkFgdVHYWZXANEHA7
11840401  5Vqgrz2M1m49PRh2eR9et7  4LOzEceyTOhRzbpebCciHR

[11840402 rows x 2 columns]


In [11]:
query = "SELECT * FROM r_artist_genre"
encoding = "latin1"
conn.text_factory = lambda x: str(x, encoding)
artist_genre_df = pd.read_sql_query(query, conn)
print(artist_genre_df)

                    genre_id               artist_id
0            detroit hip hop  4tujQJicOnuZRLiBFdp3Ou
1                     g funk  4tujQJicOnuZRLiBFdp3Ou
2               gangster rap  4tujQJicOnuZRLiBFdp3Ou
3           hardcore hip hop  4tujQJicOnuZRLiBFdp3Ou
4                    hip hop  4tujQJicOnuZRLiBFdp3Ou
...                      ...                     ...
487381                   edm  05KDKIpxshxrB9BMewaCBW
487382                   edm  6nS5roXSAGhTGr34W6n7Et
487383          samurai trap  5rcAoYM8NZrlnPZES9S8KV
487384           focus beats  0XBtKvTff8E8BIhtlxjVHv
487385  hip-hop experimental  6NgwMiz1Dwsktnn8p8yujE

[487386 rows x 2 columns]


In [12]:
r_track_artist_df = pd.merge(r_track_artist_df, artist_genre_df, on='artist_id', how='inner')
r_track_artist_df

,track_id,artist_id,genre_id
0,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,detroit hip hop
1,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,g funk
2,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,gangster rap
3,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,hardcore hip hop
4,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,hip hop
...,...,...,...
21678565,6Q42I6v0BiF8pzg9ELOX2m,5yUWxOhPOP18jZ2TwQxEtw,south african deep house
21678566,33tYADyL2aZctrvR59K1bQ,2YEnrpAWWaNRFumgde1lLH,disco house
21678567,73xL8Khrwd3OWJlBjKSpGk,2YEnrpAWWaNRFumgde1lLH,disco house
21678568,6oq0MdzLaFShOXXqs2yavp,2YEnrpAWWaNRFumgde1lLH,disco house


In [13]:
query = "SELECT * FROM artists"
encoding = "latin1"
conn.text_factory = lambda x: str(x, encoding)
artists_df = pd.read_sql_query(query, conn)
print(artists_df)

                  name                      id  popularity  followers
0               Xzibit  4tujQJicOnuZRLiBFdp3Ou          69    1193665
1         Erick Sermon  2VX0o9LDIVmKIgpnwdJpOJ          54     142007
2                J. Ro  3iBOsmwGzRKyR0vs2I61xP          45        158
3                 Tash  22qf8cJRzBjIWb2Jc4JeOr          48       3421
4           Craig Mack  4akj4uteQQrrGxhX9Rjuyf          55     161966
...                ...                     ...         ...        ...
1066026   Justice Hall  7KcttLT1Ou7ICdL3TUlc5H           0          3
1066027    Miss Malibu  68bLTKJw7zEdzEsJZS1As0           0          2
1066028     Jam'addict  0VzXnkUTwKDYznbgEvn3Ca          50        283
1066029         Sohaib  78hHMThaOCqyAGfytBUwGs           3          4
1066030  Carter Haugen  1c6IP1ctsxmIkgRToTzRXY           3          4

[1066031 rows x 4 columns]


In [14]:
artists_df.rename(columns={'id': 'artist_id'}, inplace=True)
r_track_artist_merged_df = pd.merge(r_track_artist_df, artists_df[["name", "artist_id"]], on='artist_id', how='inner')

In [15]:
r_track_artist_merged_df

,track_id,artist_id,genre_id,name
0,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,detroit hip hop,Xzibit
1,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,g funk,Xzibit
2,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,gangster rap,Xzibit
3,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,hardcore hip hop,Xzibit
4,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,hip hop,Xzibit
...,...,...,...,...
21678565,6Q42I6v0BiF8pzg9ELOX2m,5yUWxOhPOP18jZ2TwQxEtw,south african deep house,SONIDO
21678566,33tYADyL2aZctrvR59K1bQ,2YEnrpAWWaNRFumgde1lLH,disco house,Oden & Fatzo
21678567,73xL8Khrwd3OWJlBjKSpGk,2YEnrpAWWaNRFumgde1lLH,disco house,Oden & Fatzo
21678568,6oq0MdzLaFShOXXqs2yavp,2YEnrpAWWaNRFumgde1lLH,disco house,Oden & Fatzo


In [16]:
r_track_artist_merged_df

,track_id,artist_id,genre_id,name
0,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,detroit hip hop,Xzibit
1,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,g funk,Xzibit
2,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,gangster rap,Xzibit
3,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,hardcore hip hop,Xzibit
4,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,hip hop,Xzibit
...,...,...,...,...
21678565,6Q42I6v0BiF8pzg9ELOX2m,5yUWxOhPOP18jZ2TwQxEtw,south african deep house,SONIDO
21678566,33tYADyL2aZctrvR59K1bQ,2YEnrpAWWaNRFumgde1lLH,disco house,Oden & Fatzo
21678567,73xL8Khrwd3OWJlBjKSpGk,2YEnrpAWWaNRFumgde1lLH,disco house,Oden & Fatzo
21678568,6oq0MdzLaFShOXXqs2yavp,2YEnrpAWWaNRFumgde1lLH,disco house,Oden & Fatzo


In [17]:
track_and_features_df.rename(columns={'id': 'track_id'}, inplace=True)

In [18]:
grouped_artists = r_track_artist_merged_df.groupby('track_id')['name'].agg(list).reset_index()

In [19]:
grouped_artists

,track_id,name
0,0000QBRGPosiFRXKmMYnsO,[Sexteto La Playa]
1,0000gBWfr2zIFzE5tDzxca,[RattenfÃ¤nger]
2,0000uJA4xCdxThagdLkkLR,"[Cherryholmes, Cherryholmes, Cherryholmes]"
3,0001Lyv0YTjkZSqzT4WkLy,"[Half Japanese, Half Japanese, Half Japanese, ..."
4,0001QZSdENvrMx6cZXZJdo,"[Archers Of Loaf, Archers Of Loaf, Archers Of ..."
...,...,...
6558498,7zzxqMfB1N7apnN3kuos5v,"[Jonathan Butler, Jonathan Butler, Jonathan Bu..."
6558499,7zzy0bwJGb8gZ4e6qxPASG,"[Lead Belly, Lead Belly, Lead Belly, Lead Bell..."
6558500,7zzyILZOJRvpWhp7NtJJ51,"[03 Greedo, 03 Greedo, 03 Greedo, 03 Greedo, 0..."
6558501,7zzyNsSXglPKfmyawxn5KP,"[John O'Callaghan, John O'Callaghan, John O'Ca..."


In [20]:
grouped_genres = r_track_artist_merged_df.groupby('track_id')['genre_id'].agg(list).reset_index()
grouped_genres

,track_id,genre_id
0,0000QBRGPosiFRXKmMYnsO,[cha-cha-cha]
1,0000gBWfr2zIFzE5tDzxca,[ukrainian metal]
2,0000uJA4xCdxThagdLkkLR,"[bluegrass, neo-traditional bluegrass, progres..."
3,0001Lyv0YTjkZSqzT4WkLy,"[dunedin sound, experimental, experimental ind..."
4,0001QZSdENvrMx6cZXZJdo,"[alternative pop, alternative rock, emo, gbvfi..."
...,...,...
6558498,7zzxqMfB1N7apnN3kuos5v,"[smooth jazz, smooth saxophone, south african ..."
6558499,7zzy0bwJGb8gZ4e6qxPASG,"[acoustic blues, appalachian folk, blues, coun..."
6558500,7zzyILZOJRvpWhp7NtJJ51,"[cali rap, melodic rap, trap, underground hip ..."
6558501,7zzyNsSXglPKfmyawxn5KP,"[edm, pop dance, progressive house, progressiv..."


In [21]:
merged_artists_df = pd.merge(grouped_artists, grouped_genres, on='track_id', how='inner')
merged_artists_df

,track_id,name,genre_id
0,0000QBRGPosiFRXKmMYnsO,[Sexteto La Playa],[cha-cha-cha]
1,0000gBWfr2zIFzE5tDzxca,[RattenfÃ¤nger],[ukrainian metal]
2,0000uJA4xCdxThagdLkkLR,"[Cherryholmes, Cherryholmes, Cherryholmes]","[bluegrass, neo-traditional bluegrass, progres..."
3,0001Lyv0YTjkZSqzT4WkLy,"[Half Japanese, Half Japanese, Half Japanese, ...","[dunedin sound, experimental, experimental ind..."
4,0001QZSdENvrMx6cZXZJdo,"[Archers Of Loaf, Archers Of Loaf, Archers Of ...","[alternative pop, alternative rock, emo, gbvfi..."
...,...,...,...
6558498,7zzxqMfB1N7apnN3kuos5v,"[Jonathan Butler, Jonathan Butler, Jonathan Bu...","[smooth jazz, smooth saxophone, south african ..."
6558499,7zzy0bwJGb8gZ4e6qxPASG,"[Lead Belly, Lead Belly, Lead Belly, Lead Bell...","[acoustic blues, appalachian folk, blues, coun..."
6558500,7zzyILZOJRvpWhp7NtJJ51,"[03 Greedo, 03 Greedo, 03 Greedo, 03 Greedo, 0...","[cali rap, melodic rap, trap, underground hip ..."
6558501,7zzyNsSXglPKfmyawxn5KP,"[John O'Callaghan, John O'Callaghan, John O'Ca...","[edm, pop dance, progressive house, progressiv..."


In [22]:
all_df = pd.merge(track_and_features_df, merged_artists_df, on='track_id', how='inner')
all_df

,track_id,disc_number,duration_x,explicit,audio_feature_id,name_x,preview_url,track_number,popularity,is_playable,...,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,name_y,genre_id
0,4iJyoBOLtHqaGxP12qzhQI,1,198081,1,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),,12,100,NaN,...,0,0.4200,-6.181,1,0.1190,90.029999,4,0.464,"[Justin Bieber, Justin Bieber, Justin Bieber, ...","[canadian pop, pop, post-teen pop, canadian co..."
1,7lPN2DXiMsVn7XUKtOW1CS,1,242013,1,7lPN2DXiMsVn7XUKtOW1CS,drivers license,,1,100,NaN,...,10,0.1050,-8.761,1,0.0601,143.873993,4,0.132,"[Olivia Rodrigo, Olivia Rodrigo, Olivia Rodrigo]","[alt z, pop, post-teen pop]"
2,3VqeTFIvhxu3DIe4eZVzGq,1,164441,0,3VqeTFIvhxu3DIe4eZVzGq,Butter,,1,100,NaN,...,8,0.0906,-5.187,1,0.0948,109.997002,4,0.695,"[BTS, BTS]","[k-pop, k-pop boy group]"
3,67BtfxlNbhBmCDR2L2l8qd,1,137875,1,67BtfxlNbhBmCDR2L2l8qd,MONTERO (Call Me By Your Name),https://p.scdn.co/mp3-preview/fee76abaab413004...,1,99,NaN,...,8,0.3840,-6.682,0,0.1520,178.817993,4,0.758,"[Lil Nas X, Lil Nas X, Lil Nas X, Lil Nas X]","[country rap, lgbtq+ hip hop, pop rap, queer c..."
4,3BZEcbdtXQSo7OrvKRJ6mb,1,137875,1,3BZEcbdtXQSo7OrvKRJ6mb,MONTERO (Call Me By Your Name),,1,99,NaN,...,8,0.3840,-6.682,0,0.1520,178.817993,4,0.758,"[Lil Nas X, Lil Nas X, Lil Nas X, Lil Nas X]","[country rap, lgbtq+ hip hop, pop rap, queer c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9819,3WcWRr8wIkwPQOTYBWjEGf,1,244306,0,3WcWRr8wIkwPQOTYBWjEGf,Potevi fare di piÃ¹,https://p.scdn.co/mp3-preview/179164f8b2612b21...,1,67,NaN,...,4,0.0906,-8.317,1,0.0559,123.705002,4,0.162,[Arisa],[italian adult pop]
9820,1F1QmI8TMHir9SUFrooq5F,1,215384,1,1F1QmI8TMHir9SUFrooq5F,Buff Baby,https://p.scdn.co/mp3-preview/df80e5a8c9e38a8e...,2,67,NaN,...,5,0.1010,-6.047,1,0.0571,156.039993,4,0.512,"[tobi lou, tobi lou, tobi lou, tobi lou, tobi ...","[alternative r&b, chicago rap, chill r&b, pop,..."
9821,7FIkmLlwTJCK3CXrNIwXXX,1,193044,1,7FIkmLlwTJCK3CXrNIwXXX,Bis ich tot bin,,1,67,NaN,...,0,0.2150,-7.906,0,0.0816,139.011993,4,0.388,"[Jumpa, Jumpa, Zuna, Zuna]","[luxembourgian hip hop, german hip hop, albani..."
9822,69QIVIjPMnI1ABLBqh9P1b,1,260666,0,69QIVIjPMnI1ABLBqh9P1b,Twisted,https://p.scdn.co/mp3-preview/d7373f83c11623d3...,1,67,NaN,...,11,0.2480,-9.759,1,0.0466,175.848999,4,0.822,"[Keith Sweat, Keith Sweat, Keith Sweat, Keith ...","[funk, hip pop, new jack swing, quiet storm, r..."


In [23]:
all_df['name_y'] = all_df['name_y'].apply(lambda x: list(dict.fromkeys(x)))
all_df['genre_id'] = all_df['genre_id'].apply(lambda x: list(dict.fromkeys(x)))
all_df.drop(columns=['duration_y'], inplace=True)
all_df.rename(columns={'name_x': 'name', 'name_y': 'artist', 'duration_x': 'duration'}, inplace=True)
all_df

,track_id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable,...,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist,genre_id
0,4iJyoBOLtHqaGxP12qzhQI,1,198081,1,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),,12,100,NaN,...,0,0.4200,-6.181,1,0.1190,90.029999,4,0.464,"[Justin Bieber, Daniel Caesar, Giveon]","[canadian pop, pop, post-teen pop, canadian co..."
1,7lPN2DXiMsVn7XUKtOW1CS,1,242013,1,7lPN2DXiMsVn7XUKtOW1CS,drivers license,,1,100,NaN,...,10,0.1050,-8.761,1,0.0601,143.873993,4,0.132,[Olivia Rodrigo],"[alt z, pop, post-teen pop]"
2,3VqeTFIvhxu3DIe4eZVzGq,1,164441,0,3VqeTFIvhxu3DIe4eZVzGq,Butter,,1,100,NaN,...,8,0.0906,-5.187,1,0.0948,109.997002,4,0.695,[BTS],"[k-pop, k-pop boy group]"
3,67BtfxlNbhBmCDR2L2l8qd,1,137875,1,67BtfxlNbhBmCDR2L2l8qd,MONTERO (Call Me By Your Name),https://p.scdn.co/mp3-preview/fee76abaab413004...,1,99,NaN,...,8,0.3840,-6.682,0,0.1520,178.817993,4,0.758,[Lil Nas X],"[country rap, lgbtq+ hip hop, pop rap, queer c..."
4,3BZEcbdtXQSo7OrvKRJ6mb,1,137875,1,3BZEcbdtXQSo7OrvKRJ6mb,MONTERO (Call Me By Your Name),,1,99,NaN,...,8,0.3840,-6.682,0,0.1520,178.817993,4,0.758,[Lil Nas X],"[country rap, lgbtq+ hip hop, pop rap, queer c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9819,3WcWRr8wIkwPQOTYBWjEGf,1,244306,0,3WcWRr8wIkwPQOTYBWjEGf,Potevi fare di piÃ¹,https://p.scdn.co/mp3-preview/179164f8b2612b21...,1,67,NaN,...,4,0.0906,-8.317,1,0.0559,123.705002,4,0.162,[Arisa],[italian adult pop]
9820,1F1QmI8TMHir9SUFrooq5F,1,215384,1,1F1QmI8TMHir9SUFrooq5F,Buff Baby,https://p.scdn.co/mp3-preview/df80e5a8c9e38a8e...,2,67,NaN,...,5,0.1010,-6.047,1,0.0571,156.039993,4,0.512,[tobi lou],"[alternative r&b, chicago rap, chill r&b, pop,..."
9821,7FIkmLlwTJCK3CXrNIwXXX,1,193044,1,7FIkmLlwTJCK3CXrNIwXXX,Bis ich tot bin,,1,67,NaN,...,0,0.2150,-7.906,0,0.0816,139.011993,4,0.388,"[Jumpa, Zuna]","[luxembourgian hip hop, german hip hop, albani..."
9822,69QIVIjPMnI1ABLBqh9P1b,1,260666,0,69QIVIjPMnI1ABLBqh9P1b,Twisted,https://p.scdn.co/mp3-preview/d7373f83c11623d3...,1,67,NaN,...,11,0.2480,-9.759,1,0.0466,175.848999,4,0.822,[Keith Sweat],"[funk, hip pop, new jack swing, quiet storm, r..."


In [24]:
all_df.to_csv('top_songs_large.csv', index=False)

In [25]:
cursor.close()
conn.close()